- dual 4090：48GB
    - pipeline parallelism，模型按layers分两班，交替执行，一段时间只有单颗核心在计算，最终性能上肯定是下降的；
        - llama.cpp 的方案
    - tensor parallelism：每一层模型都切成两份，每张显卡只需要存半个模型，平摊显存占用；
      - 层内卡间存在数据交换，对通信速度要求高，但同一个时间，每一张显卡的核心和显存都占满了；
    - pipeline parallelism < tensor parallelism 

## 基本参考

### 1B model

- 10亿参数
- float32 vs. float16
    - float32: 4GB 内存/显存
    - float16: 2GB 内存/显存
- inference/train
    - 训练至少是 2 倍于推理时候的显存资源
        - 基本都是跟优化器（optimizer）相关
            - gradient （momentum）
            - optimizer state
            - intermediate state

### 训练技术

- 3D parallel：并行的粒度和对象；
    - tensor parallel：model layer weight matrix
        - 模型参数矩阵，1个矩阵拆解为2个小矩阵，放在两张卡上；
    - data parallel：
    - layer parallel：
        - 前面一些层放一张卡上，后边一些层也放一张卡上；
        - 模型并行：model parallel
- FSDP：Full Sharded Data Parallel，一种数据并行方法
    - ZeRO-3
- 8Bit adam optimizer / lion optimizer
- Mixed precision
- CPU offload
    - 部分的训练参数和状态放在CPU的内存里；

## bitsandbytes

- `loaded_in_8bit` 需要有 bitsandbytes 库的支撑

```
def is_bnb_available():
    return importlib.util.find_spec("bitsandbytes") is not None
```

```
loaded_in_8bit = getattr(self.model, "is_loaded_in_8bit", False)
if loaded_in_8bit and not is_bnb_available():
    raise ImportError(
        "To use Lora with 8-bit quantization, please install the `bitsandbytes` package. "
        "You can install it with `pip install bitsandbytes`."
    )
```